In [176]:
import numpy as np
import pandas as pd
import os

from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import numpy as np

import open3d as o3d

from pyquaternion import Quaternion

In [2]:
%matplotlib inline

In [3]:
%matplotlib qt

In [41]:
labels = ['LeftCarpus', 'LeftFirstMC', 'LeftFirstPP', 'LeftFirstDP', 'LeftSecondMC', 'LeftSecondPP', 'LeftSecondMP', 'LeftSecondDP', 'LeftThirdMC', 'LeftThirdPP', 'LeftThirdMP', 'LeftThirdDP', 'LeftFourthMC', 'LeftFourthPP', 'LeftFourthMP', 'LeftFourthDP', 'LeftFifthMC', 'LeftFifthPP', 'LeftFifthMP', 'LeftFifthDP']

labels_xyx = []
labels_quat = []

for label in labels:
    labels_xyx.extend([label + "_x", label + "_y", label + "_z"])
    label = label.split()[0]
    labels_quat.extend([label + "_0", label + "_1", label + "_2", label + "_3"])

In [4]:
positionFingersLeft = pd.read_excel("C:/Users/lucas/Desktop/UPC/MIT/tactile2object/manus/New Session-002.xlsx", sheet_name="ns1 positionFingersLeft")
positionFingersRight = pd.read_excel("C:/Users/lucas/Desktop/UPC/MIT/tactile2object/manus/New Session-002.xlsx", sheet_name="ns1 positionFingersRight")

In [5]:
orientationFingerLeft = pd.read_excel("C:/Users/lucas/Desktop/UPC/MIT/tactile2object/manus/New Session-002_2.xlsx", sheet_name="ns1_orientationFingerLeft")
orientationFingerLeft.columns = labels_quat

jointAngleFingersLeft = pd.read_excel("C:/Users/lucas/Desktop/UPC/MIT/tactile2object/manus/New Session-002_2.xlsx", sheet_name="ns1_jointAngleFingersLeft")
jointAngleFingersLeftXZY = pd.read_excel("C:/Users/lucas/Desktop/UPC/MIT/tactile2object/manus/New Session-002_2.xlsx", sheet_name="ns1_jointAngleFingersLeftXZY")

In [73]:
jointAngleFingersLeftXZY.shape

(10686, 57)

## Left fingers

In [44]:
x_positions = [position for position in list(positionFingersLeft.columns) if "_x" in position]
y_positions = [position for position in list(positionFingersLeft.columns) if "_y" in position]
z_positions = [position for position in list(positionFingersLeft.columns) if "_z" in position]

positions = [position.split("_")[0] for position in list(positionFingersLeft.columns)]
# Remove duplicates
positions = list(dict.fromkeys(positions))

In [45]:
positionFingersLeft_xyz = positionFingersLeft.copy()

positionFingersLeft_xyz['x'] = positionFingersLeft_xyz.apply(lambda x: 
                                                  list([x[position] for position in x_positions]), axis=1)

positionFingersLeft_xyz['y'] = positionFingersLeft_xyz.apply(lambda x: 
                                                  list([x[position] for position in y_positions]), axis=1)

positionFingersLeft_xyz['z'] = positionFingersLeft_xyz.apply(lambda x: 
                                                  list([x[position] for position in z_positions]), axis=1)

for position in positions:
    del positionFingersLeft_xyz[position + "_x"]
    del positionFingersLeft_xyz[position + "_y"]
    del positionFingersLeft_xyz[position + "_z"]

## Orientation

In [121]:
def conjugateQuaternion(vect):
    vect = vect.copy()
    vect = -vect
    vect[0] = -vect[0]
    return vect

def quaternionMultiplication(q1, q2):
    v1 = q1.copy()[1:]
    v2 = q2.copy()[1:]
    
    r0 = q1[0]*q2[0] - np.dot(v1,v2) 
    r1 = np.dot(q1[0], v2) + np.dot(q2[0], v1) + np.cross(v1, v2)
    
    r = []
    r.append(r0)
    r.extend(list(r1))
    return np.array(r)

In [106]:
def getPosition(position, frame=0):
    position_xyz = [position + i for i in ['_x','_y','_z']]
    return positionFingersLeft.loc[frame, position_xyz].to_numpy()


In [161]:
import math
 
def euler_from_quaternion(x, y, z, w):
        """
        Convert a quaternion into euler angles (roll, pitch, yaw)
        roll is rotation around x in radians (counterclockwise)
        pitch is rotation around y in radians (counterclockwise)
        yaw is rotation around z in radians (counterclockwise)
        """
        t0 = +2.0 * (w * x + y * z)
        t1 = +1.0 - 2.0 * (x * x + y * y)
        roll_x = math.atan2(t0, t1)
     
        t2 = +2.0 * (w * y - z * x)
        t2 = +1.0 if t2 > +1.0 else t2
        t2 = -1.0 if t2 < -1.0 else t2
        pitch_y = math.asin(t2)
     
        t3 = +2.0 * (w * z + x * y)
        t4 = +1.0 - 2.0 * (y * y + z * z)
        yaw_z = math.atan2(t3, t4)
     
        return roll_x, pitch_y, yaw_z # in radians

In [196]:
def getQuaternion(position, frame=0):
    data = orientationFingerLeft.loc[frame, :]
    
    return np.array([data[position + "_0"], data[position + "_1"], data[position + "_2"], data[position + "_3"]])

def getQuaternionMul(distal, proximal, frame=0):
    q1 = getQuaternion(distal, frame)
    q2 = getQuaternion(proximal, frame)

    qr = quaternionMultiplication(conjugateQuaternion(q1), q2)
    
    return qr

def getEuler(qr):
    x, y, z, w = qr
    
    return euler_from_quaternion(x, y, z, w)

distal = 'LeftSecondDP'
proximal = 'LeftSecondMP'
getQuaternionMul(distal, proximal)


array([ 0.99647428, -0.08285428, -0.01061131,  0.00785131])

# Plot hand positions

In [93]:
def set_axes_equal(ax):
    '''Make axes of 3D plot have equal scale so that spheres appear as spheres,
    cubes as cubes, etc..  This is one possible solution to Matplotlib's
    ax.set_aspect('equal') and ax.axis('equal') not working for 3D.

    Input
      ax: a matplotlib axis, e.g., as output from plt.gca().
    '''

    x_limits = ax.get_xlim3d()
    y_limits = ax.get_ylim3d()
    z_limits = ax.get_zlim3d()

    x_range = abs(x_limits[1] - x_limits[0])
    x_middle = np.mean(x_limits)
    y_range = abs(y_limits[1] - y_limits[0])
    y_middle = np.mean(y_limits)
    z_range = abs(z_limits[1] - z_limits[0])
    z_middle = np.mean(z_limits)
    
    
    # The plot bounding box is a sphere in the sense of the infinity
    # norm, hence I call half the max range the plot radius.
    plot_radius = 0.5*max([x_range, y_range, z_range])
    
    ax.set_xlim3d([x_middle - plot_radius, x_middle + plot_radius])
    # print([x_middle - plot_radius, x_middle + plot_radius])
    ax.set_ylim3d([y_middle - plot_radius, y_middle + plot_radius])
    # print([y_middle - plot_radius, y_middle + plot_radius])
    ax.set_zlim3d([z_middle - plot_radius, z_middle + plot_radius])
    # print([z_middle - plot_radius, z_middle + plot_radius])

In [216]:
tips_lengths = {
    'LeftFirst' : [0,0.02762,0], 
    'LeftSecond' : [0,0.018028,0], 
    'LeftThird' : [0,0.018916,0], 
    'LeftFourth' : [0,0.019094,0], 
    'LeftFifth' : [0,0.018384,0]
}

In [218]:
frame = 0

xs_left = positionFingersLeft_xyz.loc[frame,"x"]
ys_left = positionFingersLeft_xyz.loc[frame,"y"]
zs_left = positionFingersLeft_xyz.loc[frame,"z"]



xs = xs_left 
ys = ys_left 
zs = zs_left 

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

for i, position in enumerate(positions):
    ax.scatter(xs=xs[i], ys=ys[i], zs=zs[i], label=position, color='b')
    
fingers = ['LeftFirst', 'LeftSecond', 'LeftThird', 'LeftFourth', 'LeftFifth']

for finger in fingers:
    distal = finger + 'DP'
    proximal = finger + 'MP' if finger != 'LeftFirst' else finger + 'PP'
    
    p1 = getPosition(distal)
    q1 = Quaternion(getQuaternion(distal))
    v1 = np.array(tips_lengths[finger])
    r1 = q1.rotate(v1)
    x,y,z = r1 + p1

    ax.scatter(xs=x, ys=y, zs=z, label="NEW", color='g')



set_axes_equal(ax)
plt.axis('off')
ax.view_init(azim=153, elev=167)
#ax.legend()
plt.show()



In [212]:
getPosition('LeftFifthDP') * 2 - getPosition('LeftFifthMP')

array([-0.547078,  0.103499,  0.556152])

In [206]:
q1 = Quaternion(getQuaternion(distal))
q1

Quaternion(0.53654, -0.754379, -0.378034, 0.011302)

In [208]:
r1 = q1.rotate(v1)
r1

array([ 0.01026257, -0.00254491, -0.0150391 ])

In [210]:
finger = 'LeftFifth'
distal = finger + 'DP'
proximal = finger + 'MP' if finger != 'LeftFirst' else finger + 'PP'

getQuaternionMul(distal, proximal)

array([ 9.98730901e-01,  3.52351105e-02, -9.39067964e-04,  3.59709868e-02])